# Separating A Capella Songs into their Separated Vocal Tracks using Spleeter

In [ ]:
# Code source credit: https://github.com/deezer/spleeter/wiki/2.-Getting-started

In [1]:
# import necessary libraries
import librosa
import soundfile as sf
import numpy as np
import os
from IPython.display import Audio
import random


In [2]:
# get duration of a song
def get_duration(filename):
    y, sr = librosa.load(filename)
    duration = librosa.get_duration(y=y, sr=sr)
    return duration

# Add to csv file
def add_to_csv(mix_path, alto_path, bass_path, lead_vocal_path, soprano_path, tenor_path, vocal_percussion_path, csv_path):
    # print("adding")
    duration = get_duration(mix_path)
    with open(csv_path, 'a') as f:
        f.write(mix_path + ',' + alto_path + ',' + bass_path + ',' + lead_vocal_path + ',' + soprano_path + ',' + tenor_path + ',' + vocal_percussion_path + ',' + str(duration) + '\n')

# Put audio tracks from dataset into training csv file
def create_training_file():
    for genre in [filename for filename in os.listdir('Dataset/Jacapella') if os.path.isdir(os.path.join('Dataset/Jacapella',filename))]:
        for song in [filename for filename in os.listdir('Dataset/Jacapella/'+genre) if os.path.isdir(os.path.join('Dataset/Jacapella/'+genre,filename))]:
            mix_path = 'Dataset/Jacapella/' + genre + '/' + song + '/mixture.wav'
            alto_path = 'Dataset/Jacapella/' + genre + '/' + song + '/alto.wav'
            bass_path = 'Dataset/Jacapella/' + genre + '/' + song + '/bass.wav'
            lead_vocal_path = 'Dataset/Jacapella/' + genre + '/' + song + '/lead_vocal.wav'
            soprano_path = 'Dataset/Jacapella/' + genre + '/' + song + '/soprano.wav'
            tenor_path = 'Dataset/Jacapella/' + genre + '/' + song + '/tenor.wav'
            vocal_percussion_path = 'Dataset/Jacapella/' + genre + '/' + song + '/vocal_percussion.wav'
            add_to_csv(mix_path, alto_path, bass_path, lead_vocal_path, soprano_path, tenor_path, vocal_percussion_path, 'configs/jacapella_train.csv')
    print("Training file created") 

# sample 5 random songs from training csv to validation csv
def create_validation_file():
    with open('configs/jacapella_train.csv', 'r') as f:
        lines = f.readlines()
        random.shuffle(lines)
        for line in lines[:5]:
            with open('configs/jacapella_validation.csv', 'a') as f:
                f.write(line)
            # remove those lines from training csv
            lines.remove(line)
    print("Validation file created")

# def create_testing_file():
#     with open('configs/jacapella_train.csv', 'r') as f:
#         lines = f.readlines()
#         random.shuffle(lines)
#         for line in lines[:5]:
#             with open('configs/jacapella_test.csv', 'a') as f:
#                 f.write(line)
#             # remove those lines from training csv
#             lines.remove(line)
#     print("Testing file created")



In [3]:
# Train Spleeter
def audio_train_spleeter():
    create_training_file()
    create_validation_file()
    !spleeter train -p configs/jacapella_config.json -d '/'
    print("Audio separation trained") 


Training file created
Validation file created
INFO:spleeter:Start model training
Audio separation trained


Traceback (most recent call last):
  File "C:\Users\65900\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\65900\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\65900\anaconda3\Scripts\spleeter.exe\__main__.py", line 7, in <module>
  File "C:\Users\65900\anaconda3\lib\site-packages\spleeter\__main__.py", line 267, in entrypoint
    spleeter()
  File "C:\Users\65900\anaconda3\lib\site-packages\typer\main.py", line 214, in __call__
    return get_command(self)(*args, **kwargs)
  File "C:\Users\65900\anaconda3\lib\site-packages\click\core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "C:\Users\65900\anaconda3\lib\site-packages\click\core.py", line 782, in main
    rv = self.invoke(ctx)
  File "C:\Users\65900\anaconda3\lib\site-packages\click\core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "C:\Users\

In [ ]:
audio_train_spleeter()

In [ ]:
!spleeter separate -p configs/jacapella_config.json -o output Dataset/Jacapella/Pop/1/mixture.wav